In [1]:
import sys
import nltk
import sklearn
import pandas as pd
import numpy as np

print('Python: {}'.format(sys.version))
print('NLTKL: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))

Python: 3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]
NLTKL: 3.8.1
Scikit-learn: 1.5.0
Pandas: 2.1.2
Numpy: 1.24.2


## 1. Load the Dataset

In [2]:
df = pd.read_table('SMSSpamCollection',header = None, encoding='utf8')

In [3]:
# print useful info about the dataset
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
# check the class distribution
classes = df[0]
print(classes.value_counts())

0
ham     4825
spam     747
Name: count, dtype: int64


## 2. Preprocess the data

In [7]:
# convert class to binary values, 0 = ham, 1 = spam
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(Y[:10])
print(classes[:10])

[0 0 1 0 0 1 0 0 1 1]
0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: 0, dtype: object


In [8]:
# store the sms msg data
text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


Instead of building your own REGEX, you can use: https://regexlib.com/?AspxAutoDetectCookieSupport=1

In [22]:
# use regular expressions to replace email addresses, URLs, phone numbers, other numbers

# Replace email addresses with 'email'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 'emailaddress',regex=True)

# Replace URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                  'webaddress',regex=True)

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr',regex=True)
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr',regex=True)

In [23]:
# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ',regex=True)

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ',regex=True)

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '',regex=True)

In [24]:
# change words to lower case
processed = processed.str.lower()
print (processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [25]:
# remove stop words from text messages
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))

In [26]:
# remove word stem using Porter Stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: " ".join(ps.stem(term) for term in x.split()))

In [27]:
print(processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    numbrnd time tri numbr contact u u numbr pound...
5568                              ü b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


In [55]:
from nltk.tokenize import word_tokenize

# creating a bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

all_words = nltk.FreqDist(all_words)

In [56]:
# print the total number words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 6564
Most common words: [('numbr', 2975), ('u', 1207), ('call', 679), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266), ('like', 261)]


In [63]:
# use the 1500 most common words as features
sorted_words = [word for word, frequency in sorted(all_words.items(), key=lambda x: x[1], reverse=True)]
word_features = sorted_words[:1500]

6564
1500


In [67]:
# define a find_feature function
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

# Lets see an example
features = find_features(processed[0])
for key,value in features.items():
    if value == True:
        print(key, value)


go True
got True
n True
great True
wat True
e True
world True
point True
avail True
crazi True
bugi True
la True
cine True


In [74]:
# find features for all messages
messages = zip(processed, Y)
messages = list(messages)

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS msgs
featuresets=[(find_features(text),label) for (text,label) in messages]

In [75]:
# split training and testing data sets using sklearn
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state = seed, shuffle = True)

In [77]:
print('Training: {}'.format(len(training)))
print('Testing: {}'.format(len(testing)))

Training: 4179
Testing: 1393


## 3. Scikit-Learn Classifiers with NLTK

In [78]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [83]:
# define models to train
names = ['K Nearest Neighbours',
         'Decision Tree',
         'Random Forest',
         'Logistic Regression',
         'SGD Classifier',
         'Naive Bayes',
         'SVM Linear']

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names,classifiers)

In [84]:
# wrap models in NLTK
from nltk.classify.scikitlearn import SklearnClassifier

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing) * 100
    print('{}: Accuracy: {}'.format(name,accuracy))

K Nearest Neighbours: Accuracy: 95.908111988514
Decision Tree: Accuracy: 96.62598707824839
Random Forest: Accuracy: 98.49246231155779
Logistic Regression: Accuracy: 98.77961234745155
SGD Classifier: Accuracy: 98.56424982053123
Naive Bayes: Accuracy: 97.5592246949031
SVM Linear: Accuracy: 98.27709978463747


In [87]:
# ensemble method - Voting classifier
from sklearn.ensemble import VotingClassifier

# define models to train
names = ['K Nearest Neighbours',
         'Decision Tree',
         'Random Forest',
         'Logistic Regression',
         'SGD Classifier',
         'Naive Bayes',
         'SVM Linear']

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names,classifiers)

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators=list(models), voting = 'hard', n_jobs=-1))
nltk_ensemble.train(training)

accuracy = nltk.classify.accuracy(nltk_ensemble, testing) * 100
print('Ensemble Method Accuracy: {}'.format(accuracy))

Ensemble Method Accuracy: 98.63603732950466


In [88]:
# make class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

In [89]:
# print a confusion matrix and a classificatino report
print(classification_report(labels,prediction))

pd.DataFrame(
    confusion_matrix(labels,prediction),
    index = [['actual','actual'], ['ham','spam']],
    columns = [['predicted','predicted'],['ham','spam']]
)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1223
           1       0.98      0.91      0.94       170

    accuracy                           0.99      1393
   macro avg       0.98      0.95      0.97      1393
weighted avg       0.99      0.99      0.99      1393



predicted     
                  ham spam
actual ham       1220    3
       spam        16  154